In [ ]:
from datetime import datetime
import load_csv
import matplotlib.pyplot as plt  # type: ignore
import numpy as np  # type: ignore
import pandas as pd  # type: ignore
import seaborn as sns  # type: ignore

# Load Data

In [ ]:
area_data = load_csv.load_area_data_2018(
    "../../data/area/statistical-area-2-2018-generalised.csv"
)

In [ ]:
telco_data = load_csv.load_telco_data("../../data/telco/pop_data_2020-04-01.dat")

In [ ]:
data_all = pd.merge(area_data[["region_code", "region_name", "area"]], telco_data, on="region_code")

In [ ]:
data_all['density'] = data_all['count'] / data_all['area']

In [ ]:
data_all

In [ ]:
data_recent = data_all[
    data_all["time"] >= datetime(2020, 2, 1)
]

In [ ]:
data_recent

# Quick checks

### Total counts

In [ ]:
data = data_all

In [ ]:
data_summed = data.groupby('time').sum()
data_summed

In [ ]:
sns_plot = sns.lineplot(
    data = data_summed,
    x=data_summed.index,
    y='count',
)

sns_plot.get_figure().autofmt_xdate()

Why are there more phones than people?
- Tourists?
Maybe, there is a decline at the end of summer.

- People with two phones?


What are the occaisional drops? There are a few days when the total count is less than 4.6 million.

In [ ]:
data_summed[data_summed['count'] < 4600000]

### Can seeing commuting and holidays

In [ ]:
data_subset = data_recent[
    np.logical_or.reduce(
        (
            data_recent["region_name"] == "Auckland-University",
            # data["region_name"] == "Queen Street",
            data_recent["region_name"] == "Balmoral",
            data_recent["region_name"] == "Mount Maunganui North",
        )
    )
]

In [ ]:
sns_plot = sns.lineplot(
    data=data_subset,
    x="time",
    y="count",
    hue="region_name",
)
sns_plot.set_xlim(right=datetime(2020, 2, 29))
sns_plot.get_figure().autofmt_xdate()

plt.show()

This makes sense.

- Five spikes a week when people go to work - except Waitangi day.

- Auckland-University and Balmoral are anti-correlated.

- Spikes in Mount Manuganui evenings, weekends and holidays.

# Can see major gatherings

In [ ]:
data_subset = data_all[
    (
        (data_all["region_name"] == "Eden Park") |
        (data_all["region_name"] == "Ellerslie Central") |
        (data_all["region_name"] == "Westmere South-Western Springs") |
        (data_all["region_name"] == "Riccarton Racecourse") |
        False
    ) &
    (data_all['time'] >= datetime(2019, 1, 1)) &
    (data_all['time'] <= datetime(2019, 4, 4))
]

In [ ]:
sns_plot = sns.lineplot(
    data=data_subset,
    x="time",
    y="count",
    hue="region_name",
)
sns_plot.get_figure().autofmt_xdate()

plt.show()

### Overall densities

In [ ]:
date = datetime(2020, 2, 26, 14)
sns_plot = sns.distplot(
    data_all[data_all['time'] == date]['density'],
    kde=False,
)

sns_plot.set_yscale('log')
sns_plot.set_ylabel('count')
sns_plot.set_title(date)

In [ ]:
data_all[
#     (data_all['density'] > 100_000) &
    (data_all['time'] == datetime(2020, 2, 26, 14))
].sort_values('density', ascending=False)